## Wedge Project Part 2 - Creating a  Sample of Owners

In [98]:
import os
import re
import random

import pandas as pd
import numpy as np
import pandas_gbq
from random import sample

from google.cloud import bigquery
from google.oauth2 import service_account

### Connect to GBQ

In [43]:

service_path = r"C:\Users\Sam Beighle\Documents\ADA"
service_file = r'\wedge-project-sam-beighle-3df5c228fad7.json'
gbq_proj_id = 'wedge-project-sam-beighle'

# And this should stay the same. 
private_key =service_path + service_file
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)
#let's make sure it works

for item in client.list_datasets() : 
    print(item.full_dataset_id)

wedge-project-sam-beighle:new_dataset_id
wedge-project-sam-beighle:owner_products
wedge-project-sam-beighle:test
wedge-project-sam-beighle:wedge_project_beighle


In [44]:
service_path = r"C:\Users\Sam Beighle\Documents\ADA"
service_file = r'C:\Users\Sam Beighle\Documents\ADA\wedge-project-sam-beighle-3df5c228fad7.json'

# Load the service account credentials
credentials = service_account.Credentials.from_service_account_file(private_key)

# Create a BigQuery client
client = bigquery.Client(credentials=credentials)

# Get the email address of the service account
service_account_email = credentials.service_account_email

print(f"Using service account: {service_account_email}")

Using service account: big-query@wedge-project-sam-beighle.iam.gserviceaccount.com


### Query and Creating CSV

In [75]:
owner_sample = []

query = """
        SELECT DISTINCT(card_no)
        FROM `wedge-project-sam-beighle.wedge_project_beighle.transArchive_2*`
        WHERE card_no != 3"""

results = client.query(
    query,
)


for row in results :
    owner_sample.append(row.values()[0])




In [ ]:
for row in results:
    print(row)


In [83]:
random.seed(10)
random_owner_sample = random.choices(owner_list, k=200)

In [92]:
query = """     
SELECT card_no, *
FROM `wedge-project-sam-beighle.wedge_project_beighle.transArchive_201004_201006_clean`
WHERE card_no in (
"""
query = query +",".join([str(element) for element in random_owner_sample])+")"

In [ ]:
print(query)

In [ ]:
print(random_owner_sample)

In [96]:
sample__of_owners = client.query(query).to_dataframe()
sample__of_owners.to_csv("wedge_project_owner_sample_final.csv")

In [82]:
print(query[:100])


        SELECT DISTINCT(card_no)
        FROM `wedge-project-sam-beighle.wedge_project_beighle.tran


### Output to check if worked, card_no as first column

In [97]:
import pandas as pd

sample_of_owners = pd.read_csv("wedge_project_owner_sample_final.csv")

print(sample_of_owners.head())


   Unnamed: 0  card_no                   datetime  register_no  emp_no  \
0           0    35228  2010-05-08 11:35:49+00:00           16      54   
1           1    44417  2010-04-04 12:25:24+00:00            6      44   
2           2    44213  2010-06-18 11:17:56+00:00            9      50   
3           3    16825  2010-06-08 15:16:07+00:00            4      87   
4           4    20410  2010-06-12 13:14:59+00:00            4      46   

   trans_no upc  description trans_type trans_subtype  ... batchHeaderID  \
0        79   0       Change          T            CA  ...           NaN   
1       142   0       Change          T            CA  ...           NaN   
2        38   0  Credit Card          T            CC  ...           NaN   
3        11   0         Cash          T            CA  ...           NaN   
4        31   0       Change          T            CA  ...           NaN   

   local  organic  display  receipt  card_no_1  store  branch  match_id  \
0      0      NaN      